### Load Key Libraries

In [1]:
import numpy as np
import tensorflow as tf
import keras
import cv2
import pandas as pd
import random

Using TensorFlow backend.


### Load Data

In [10]:
## Load Labels
import csv
labels = [];
with open('data/label.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        labels.append(row[0])

In [12]:
labels = [int(label) for label in labels]

In [13]:
from collections import Counter
c = Counter(labels)
print(c)

Counter({0: 7331, 7: 4295, 2: 3957, 12: 2033, 10: 1974, 8: 1568, 21: 852, 5: 765, 4: 631, 18: 598, 9: 473, 16: 401, 6: 259, 3: 172, 11: 149, 14: 132, 13: 107, 17: 104, 15: 72, 1: 39, 20: 20, 19: 1})


In [15]:
### Put Files Into Correct Folders, Only do it once
'''
import os
import shutil

count = 0;
for i in range(1, 26201):
    path1 = 'img/img' + '%05d' %i + '.jpg'
    if (os.path.isfile(path1)):
        if (i%10 < 8):
            path2 = 'train_img/' + '%d' %labels[count] + '/%05d' %i + '.jpg'
        else:
            path2 = 'valid_img/' + '%d' %labels[count] + '/%05d' %i + '.jpg'
        
        shutil.move(path1, path2)
        count += 1
'''

### Preprocessing

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def randombrightness(img):
    return img * np.random.uniform(0.95, 1.05)

datagen = ImageDataGenerator(
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 90,
    shear_range=0.1,
    rescale = 1./255,
    zoom_range = [0.95, 1.05],
    channel_shift_range = 15,
    preprocessing_function = randombrightness,
    fill_mode='nearest'
)

### Build The Model From Scratch

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(filters = 32, kernel_size = 3, padding='valid', activation='relu', input_shape=(440,440,3)),
    MaxPooling2D(2,2,'valid'),
    Conv2D(filters = 64, kernel_size = 3, padding='valid', activation='relu'),
    MaxPooling2D(2,2,'valid'),
    Conv2D(filters = 128, kernel_size = 3, padding='same', activation='relu'),
    MaxPooling2D(2,2,'same'),
    Conv2D(filters = 256, kernel_size = 3, padding='same', activation='relu'),
    MaxPooling2D(2,2,'same'),
    Conv2D(filters = 512, kernel_size = 3, padding='same', activation='relu'),
    MaxPooling2D(2,2,'same'),
    Conv2D(filters = 1024, kernel_size = 3, padding='same', activation='relu'),
    MaxPooling2D(2,2,'same'),
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(22, activation='softmax')
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 438, 438, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 219, 219, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 217, 217, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 108, 108, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 54, 54, 256)       295168    
__________

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
## random crop
def random_crop(img, random_crop_size):
    dx, dy = random_crop_size
    height, width = img.shape[0], img.shape[1]
    x = np.random.randint(50, width-dx+1-50)
    y = np.random.randint(50, height-dy+1-50)
    return img[y:(y+dy), x:(x+dx)]

def fix_crop(img, random_crop_size):
    dx, dy = random_crop_size
    x = 108
    y = 108
    return img[y:(y+dy), x:(x+dx)]

def crop_generator(batches, crop_length):
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)
        
def crop_generator2(batches, crop_length):
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = fix_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [3]:
train_batches = datagen.flow_from_directory(
    'train_img',
    target_size=(440,440),
    batch_size=32
)
#train_crops = crop_generator(train_batches, 224)

Found 20738 images belonging to 22 classes.


In [4]:
datagen_v = ImageDataGenerator(rescale=1./255)

In [5]:
valid_batches = datagen_v.flow_from_directory(
    'valid_img',
    target_size=(440, 440),
    batch_size=32
)
#valid_crops = crop_generator2(valid_batches, 224)

Found 5195 images belonging to 22 classes.


In [15]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='mymodel/weights.best.fromScratch.hdf5',
                              verbose=1, save_best_only=True)
model.fit_generator(
    train_batches,
    steps_per_epoch = int(20738 / 32),
    epochs=20,
    validation_data = valid_batches,
    validation_steps = int(5195 / 32),
    callbacks = [checkpointer],
    verbose=1
)

Epoch 1/20
648/648 [==============================] - 743s - loss: 1.8473 - acc: 0.3974 - val_loss: 1.2574 - val_acc: 0.5565
Epoch 2/20
648/648 [==============================] - 736s - loss: 1.1401 - acc: 0.5890 - val_loss: 0.8947 - val_acc: 0.6669
Epoch 3/20
648/648 [==============================] - 736s - loss: 0.9120 - acc: 0.6667 - val_loss: 0.8084 - val_acc: 0.6917
Epoch 4/20
648/648 [==============================] - 732s - loss: 0.8006 - acc: 0.7081 - val_loss: 0.6819 - val_acc: 0.7395
Epoch 5/20
648/648 [==============================] - 733s - loss: 0.7241 - acc: 0.7426 - val_loss: 0.5934 - val_acc: 0.7840
Epoch 6/20
648/648 [==============================] - 732s - loss: 0.6516 - acc: 0.7687 - val_loss: 0.5954 - val_acc: 0.7918
Epoch 7/20
648/648 [==============================] - 733s - loss: 0.6049 - acc: 0.7832 - val_loss: 0.5806 - val_acc: 0.8053
Epoch 8/20
648/648 [==============================] - 732s - loss: 0.5806 - acc: 0.7897 - val_loss: 0.6487 - val_acc: 0.7465


### Continue Training

In [ ]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='mymodel/weights.best.fromScratch.hdf5',
                              verbose=1, save_best_only=True)
model = load_model('mymodel/weights.best.fromScratch.hdf5')
model.fit_generator(
    train_batches,
    steps_per_epoch = int(20738 / 32),
    epochs=50,
    validation_data = valid_batches,
    validation_steps = int(5195 / 32),
    callbacks = [checkpointer],
    verbose=1
)

Epoch 1/50
648/648 [==============================] - 778s - loss: 0.3978 - acc: 0.8567 - val_loss: 0.3758 - val_acc: 0.8607
Epoch 2/50
648/648 [==============================] - 767s - loss: 0.4078 - acc: 0.8536 - val_loss: 0.3968 - val_acc: 0.8594
Epoch 3/50
648/648 [==============================] - 767s - loss: 0.4107 - acc: 0.8542 - val_loss: 0.3636 - val_acc: 0.8698
Epoch 4/50
648/648 [==============================] - 766s - loss: 0.4090 - acc: 0.8521 - val_loss: 0.3915 - val_acc: 0.8573
Epoch 5/50
648/648 [==============================] - 769s - loss: 0.3971 - acc: 0.8545 - val_loss: 0.3797 - val_acc: 0.8600
Epoch 6/50
648/648 [==============================] - 769s - loss: 0.3992 - acc: 0.8556 - val_loss: 0.3431 - val_acc: 0.8666
Epoch 7/50
648/648 [==============================] - 767s - loss: 0.3968 - acc: 0.8552 - val_loss: 0.3901 - val_acc: 0.8584
Epoch 8/50
648/648 [==============================] - 770s - loss: 0.4018 - acc: 0.8551 - val_loss: 0.3754 - val_acc: 0.8681


648/648 [==============================] - 765s - loss: 0.3870 - acc: 0.8617 - val_loss: 0.3827 - val_acc: 0.8677
Epoch 34/50
648/648 [==============================] - 765s - loss: 0.3862 - acc: 0.8632 - val_loss: 0.4186 - val_acc: 0.8636
Epoch 35/50
648/648 [==============================] - 765s - loss: 0.4088 - acc: 0.8558 - val_loss: 0.3643 - val_acc: 0.8727
Epoch 36/50
648/648 [==============================] - 764s - loss: 0.3831 - acc: 0.8614 - val_loss: 0.3760 - val_acc: 0.8677
Epoch 37/50
648/648 [==============================] - 768s - loss: 0.3842 - acc: 0.8627 - val_loss: 0.3483 - val_acc: 0.8774
Epoch 38/50
648/648 [==============================] - 767s - loss: 0.3821 - acc: 0.8651 - val_loss: 0.3686 - val_acc: 0.8716
Epoch 39/50
648/648 [==============================] - 767s - loss: 0.3844 - acc: 0.8624 - val_loss: 0.3794 - val_acc: 0.8644
Epoch 40/50
648/648 [==============================] - 768s - loss: 0.3866 - acc: 0.8607 - val_loss: 0.3720 - val_acc: 0.8602
Epoc

## Conclusion: Build From Scratch has Best Result 88%

### Try ResNet 50 Fine Tuning

In [29]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    horizontal_flip = True,
    rotation_range = 50,
    shear_range=0.1,
    fill_mode='nearest'
)
train_generator = datagen.flow_from_directory(
    'train_img',
    target_size=(224,224),
    batch_size=1
)

Found 20738 images belonging to 22 classes.


In [30]:
datagen_v = ImageDataGenerator()
valid_generator = datagen_v.flow_from_directory(
    'valid_img',
    target_size=(224,224),
    batch_size=1
)

Found 5195 images belonging to 22 classes.


In [31]:
from keras.applications import resnet50
model2 = resnet50.ResNet50(include_top=False, weights='imagenet')

In [32]:
bottleneck_features_train = model2.predict_generator(
    train_generator,
    20738,
)
np.save('bottleneck_features_train.npy', bottleneck_features_train)

In [33]:
len(bottleneck_features_train)

20738

In [34]:
bottleneck_features_valid = model2.predict_generator(
    valid_generator,
    5195,
)
np.save('bottleneck_features_valid.npy', bottleneck_features_valid)

In [35]:
len(bottleneck_features_valid)

5195

In [36]:
from keras.utils.np_utils import to_categorical
train_data = np.load('bottleneck_features_train.npy')
train_labels = train_generator.classes
train_labels = to_categorical(train_labels[0:len(bottleneck_features_train)], num_classes=22)

In [37]:
valid_data = np.load('bottleneck_features_valid.npy')
valid_labels = valid_generator.classes
valid_labels = to_categorical(valid_labels[0:len(bottleneck_features_valid)], num_classes=22)

In [38]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

model2 = Sequential([
    Flatten(input_shape = train_data.shape[1:]),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(22, activation='softmax')
])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
model2.fit(
    train_data,
    train_labels,
    epochs = 20,
    batch_size = 32,
    validation_data = (valid_data, valid_labels),
    verbose=1
)

Train on 20738 samples, validate on 5195 samples
Epoch 1/20
20738/20738 [==============================] - 4s - loss: 2.2878 - acc: 0.2806 - val_loss: 2.2796 - val_acc: 0.2914
Epoch 2/20
20738/20738 [==============================] - 4s - loss: 2.2864 - acc: 0.2806 - val_loss: 2.2521 - val_acc: 0.2914
Epoch 3/20
20738/20738 [==============================] - 4s - loss: 2.2790 - acc: 0.2804 - val_loss: 2.2471 - val_acc: 0.2914
Epoch 4/20
20738/20738 [==============================] - 4s - loss: 2.2762 - acc: 0.2805 - val_loss: 2.2470 - val_acc: 0.2914
Epoch 5/20
20738/20738 [==============================] - 4s - loss: 2.2732 - acc: 0.2805 - val_loss: 2.2531 - val_acc: 0.2912
Epoch 6/20
20738/20738 [==============================] - 4s - loss: 2.2685 - acc: 0.2806 - val_loss: 2.2541 - val_acc: 0.2914
Epoch 7/20
20738/20738 [==============================] - 4s - loss: 2.2662 - acc: 0.2805 - val_loss: 2.2469 - val_acc: 0.2914
Epoch 8/20
20738/20738 [==============================] - 4s -

## Fine Tune ResNet 50 Completely Fails

### Retrain ResNet 50

In [70]:
from keras.applications import resnet50
from keras.layers import Input
model2 = resnet50.ResNet50(include_top=True, weights=None, input_tensor=Input(shape=(224,224,3)), classes=22)

In [63]:
#model2.summary()

In [65]:
#If you want to use your own top
'''from keras.models import Model
x = model2.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
x = Dense(22)(x)
model2 = Model(model2.input, x)
'''

In [71]:
model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_8 (ZeroPadding2D) (None, 230, 230, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256                                          
___________________________________________________________________________________________

In [72]:
datagen = ImageDataGenerator(
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    horizontal_flip = True,
    rotation_range = 50,
    shear_range=0.1,
    rescale = 1./255,
    fill_mode='nearest'
)
train_generator = datagen.flow_from_directory(
    'train_img',
    target_size=(224,224),
    batch_size=32
)
datagen_v = ImageDataGenerator(
    rescale = 1./255
)
valid_generator = datagen_v.flow_from_directory(
    'valid_img',
    target_size=(224,224),
    batch_size=32
)

Found 20738 images belonging to 22 classes.
Found 5195 images belonging to 22 classes.


In [73]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='mymodel/weights.best.ResNet50.hdf5',
                              verbose=1, save_best_only=True)
model2.fit_generator(
    train_generator,
    steps_per_epoch = int(20738 / 32),
    epochs=20,
    validation_data = valid_generator,
    validation_steps = int(5195 / 32),
    callbacks = [checkpointer],
    verbose=1
)

Epoch 1/20
648/648 [==============================] - 823s - loss: 0.6279 - acc: 0.7722 - val_loss: 4.4826 - val_acc: 0.1691
Epoch 2/20
648/648 [==============================] - 780s - loss: 0.5845 - acc: 0.7888 - val_loss: 2.3989 - val_acc: 0.3322
Epoch 3/20
648/648 [==============================] - 778s - loss: 0.5642 - acc: 0.7962 - val_loss: 11.6199 - val_acc: 0.1036
Epoch 4/20
648/648 [==============================] - 781s - loss: 0.6076 - acc: 0.7821 - val_loss: 2.1515 - val_acc: 0.4249
Epoch 5/20
648/648 [==============================] - 778s - loss: 0.5355 - acc: 0.8032 - val_loss: 15.3001 - val_acc: 0.0294
Epoch 6/20
648/648 [==============================] - 778s - loss: 0.5077 - acc: 0.8155 - val_loss: 2.7377 - val_acc: 0.3796
Epoch 7/20
648/648 [==============================] - 780s - loss: 0.5024 - acc: 0.8185 - val_loss: 1.1013 - val_acc: 0.6438
Epoch 8/20
648/648 [==============================] - 778s - loss: 0.5147 - acc: 0.8125 - val_loss: 2.8905 - val_acc: 0.429

In [ ]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='mymodel/weights.best.ResNet50.hdf5',
                              verbose=1, save_best_only=True)
model2 = load_model('mymodel/weights.best.ResNet50.hdf5')
model2.fit_generator(
    train_generator,
    steps_per_epoch = int(20738 / 32),
    epochs=20,
    validation_data = valid_generator,
    validation_steps = int(5195 / 32),
    callbacks = [checkpointer],
    verbose=1
)

Epoch 1/20
648/648 [==============================] - 845s - loss: 0.4543 - acc: 0.8355 - val_loss: 13.7865 - val_acc: 0.0389
Epoch 2/20
272/648 [===========>..................] - ETA: 417s - loss: 0.4476 - acc: 0.8356